[View in Colaboratory](https://colab.research.google.com/github/Angordil/Analisis-de-registros-de-EEG-mediante-redes-neuronales/blob/master/Cross_Validation_Conv2D.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.python.keras import utils
from tensorflow.python.keras.models import Sequential, save_model, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.applications import imagenet_utils
from scipy.io import loadmat
import itertools
from sklearn.metrics import confusion_matrix

In [0]:
def conv(X_training, y_training1hot, X_validation, y_validation1hot):

  num_classes = 6

  model = Sequential()

  model.add(Conv2D(16, (3,3), input_shape = (124,32,1)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(128))
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    # Train model
    
  history = model.fit(X_training, y_training1hot, epochs = 84, validation_data = (X_validation, y_validation1hot), shuffle = True, verbose = 1)

  #model.save_weights('drive/TFG/Trial only with subject 1/Weights/weightsVoteModel'+str(subj_num+1)+'.hdf5')

  return model, history

def data_preparation(subj_num):
  # Load data 

  subject = loadmat('drive/TFG/data/S'+str(subj_num+1)+'.mat')

  # Inspect the EEG data for the first trial of the first subject:

  X_training = subject['X_2D']
  
  N = 32
  
  electrodes = 124
  
  # ### Data Preparation

  y_training = subject['categoryLabels'][0]

  # Validation set will be defined when model.fit is done. It'll be a 10% of the training data

  ## 1-Hot Encode Target Output Labels
  num_classes = 6

  y_training1hot = utils.to_categorical(y_training - 1 , num_classes)

  print y_training1hot.shape

  # Reshape training set

  X_training = np.reshape(X_training,(-1, electrodes, N,1)) 
  
  return X_training, y_training1hot, y_training

def conf_matrix(y_validation, X_validation, model):
  # ### Model Quality Check: Confusion Matrix
  
  y_validation_predictions = model.predict(X_validation, verbose = 1)

  cnf_matrix = confusion_matrix(y_validation - 1, np.argmax(y_validation_predictions, axis = 1))

  plt.figure()
  class_names = ['Human Body', 'Human Face', 'Animal Body', 'Animal Face', 'Fruit Vegetable', 'Inanimate Object']
  plot_confusion_matrix(cnf_matrix, classes = class_names, title = 'Confusion matrix ')
  
def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting 'normalize = True'
    """
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    if normalize:
      cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
      print('Normalized confusion matrix')
    else:
      print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j], horizontalalignment = 'center', color = 'white' if cm[i, j] > thresh else 'black')

      plt.tight_layout()
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      

In [0]:
# Data preparation and concatenation
X_train, y_train1hot, y_train = data_preparation(0)

for subj in range(1,10):
  X_training, y_training1hot, y_training = data_preparation(subj)
  X_train = np.append(X_train, X_training, axis = 0)
  y_train1hot = np.append(y_train1hot, y_training1hot, axis = 0)
  y_train = np.append(y_train, y_training, axis = 0)
  print('Subj ' + str(subj) + ' added')

random_index_training = np.random.randint(len(X_train), size = len(X_train))

In [0]:
# Simple validation --> one loops, 10% as validation
random_index_training = np.random.randint(len(X_train), size = len(X_train))
start = 0 

fin = start+5186
if fin > len(X_train):
  fin = len(X_train)
  
Xval = X_train[random_index_training[start:fin]]
yval1hot = y_train1hot[random_index_training[start:fin]]
yval = y_train[random_index_training[start:fin]]
if start == 0:
  Xtrain = X_train[random_index_training[fin:]]
  ytrain1hot = y_train1hot[random_index_training[fin:]]
else:
  Xtrain = X_train[random_index_training[0:start]]
  x = X_train[random_index_training[fin:]]
  Xtrain = np.append(Xtrain, x, axis = 0)
  
  ytrain1hot = y_train1hot[random_index_training[0:start]]
  y = y_train1hot[random_index_training[fin:]]
  ytrain1hot = np.append(ytrain1hot, y, axis = 0)
  
model, history = conv(Xtrain, ytrain1hot, Xval, yval1hot)
prediction = model.predict(Xval, verbose = 0)
val = np.argmax(prediction, axis = 1) 


## Compare predicted classes with actual classes
correct = 0
for i in range(0, len(yval)):
  if(yval[i]==(val[i]+1)):
    correct = correct +1
  else: correct = correct

accuracy = correct*100/len(yval)
print (str(accuracy) + ' % - loop ' )

start = fin

conf_matrix(yval, Xval, model)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy over time')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Cross Entropy Loss over Time')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show

In [0]:
# Cross validation --> ten loops, 10% as validation
random_index_training = np.random.randint(len(X_train), size = len(X_train))
start = 0 

for loop in range(0,10):
  fin = start+5186
  if fin > len(X_train):
    fin = len(X_train)
    
  Xval = X_train[random_index_training[start:fin]]
  yval1hot = y_train1hot[random_index_training[start:fin]]
  yval = y_train[random_index_training[start:fin]]
  if start == 0:
    Xtrain = X_train[random_index_training[fin:]]
    ytrain1hot = y_train1hot[random_index_training[fin:]]
  else:
    Xtrain = X_train[random_index_training[0:start]]
    x = X_train[random_index_training[fin:]]
    Xtrain = np.append(Xtrain, x, axis = 0)
    
    ytrain1hot = y_train1hot[random_index_training[0:start]]
    y = y_train1hot[random_index_training[fin:]]
    ytrain1hot = np.append(ytrain1hot, y, axis = 0)
    
  model, history = conv(Xtrain, ytrain1hot, Xval, yval1hot)
  prediction = model.predict(Xval, verbose = 0)
  val = np.argmax(prediction, axis = 1) 

  
  ## Compare predicted classes with actual classes
  correct = 0
  for i in range(0, len(yval)):
    if(yval[i]==(val[i]+1)):
      correct = correct +1
    else: correct = correct

  accuracy = correct*100/len(yval)
  print (str(accuracy) + ' % - loop ' + str(loop+1))
  if loop == 0:
    Acc = [accuracy]
    hist = [history]
  else:
    Acc = np.append(Acc, [accuracy], axis = 0)
    hist = np.append(hist, [history], axis = 0)
    
  start = fin
  
  conf_matrix(yval, Xval, model)
  
print("Final accuracy: " + str(Acc) + "%")
